In [1]:
import cv2
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model('./model/model.h5')

In [3]:
cameraNumber = 1
width = 320
height = 240

cap = cv2.VideoCapture(cameraNumber)
cap.set(3, width)
cap.set(4, height)

def Crop(image):
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    cv2.imshow("img hsv", img_hsv)
    
    lower = np.array([0, 50, 50])
    upper = np.array([10, 255, 255])
    mask = cv2.inRange(img_hsv, lower, upper)
    image[np.where(mask == 0)] = 0
    image[np.where(mask != 0)] = 255
    cv2.imshow("mask", image)
    
    _, thresh = cv2.threshold(mask, 70, 255, 0)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    rect = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(rect)
    img_rect = image.copy()
    cv2.drawContours(img_rect, contours, -1, (255, 0, 0), 2)
    cv2.rectangle(img_rect, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.imshow("img rect", img_rect)
    
    img_crop = image[y:y + h, x:x + w]
    cv2.imshow("img crop", img_crop)
    
    img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
    return img_crop

def PreProcessing(image):
    image = Crop(image)
    image = cv2.resize(image, (28, 28))
    image = image.reshape(-1, 28, 28, 1)
    return image

def Predict(model, image):
    image = PreProcessing(image)
    classIndex = int(model.predict_classes(image))
    predictions = model.predict(image)
    probVal = np.amax(predictions)
    return (classIndex, probVal)

while True:
    try:
        _, frame = cap.read()
        
        classIndex, probVal = Predict(model, frame.copy())
        
        cv2.putText(frame, f"{classIndex} {probVal}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.imshow("Camera", frame)
        
        keyPress = cv2.waitKey(1)
        if keyPress % 256 == 32:
            cv2.imwrite("./images/sample.png", frame)
        elif keyPress & 0xFF == ord('q'):
            break
    except:
        pass
        
cap.release()
cv2.destroyAllWindows()